#### Build baseline model notebook for CP1 using Logistic Regression, with hyper-parameter tuning (regularization parameter), L1 and L2 regularization.  

In [6]:
%matplotlib inline
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

In [7]:
df_agb = pd.read_csv('age_gender_bkts.csv')
#Convert 100+ into a bin.
df_agb['age_bucket'] = df_agb['age_bucket'].apply(lambda x: '100-104' if x == '100+' else x)
#Define mean_age feature
df_agb['mean_age'] = df_agb['age_bucket'].apply(lambda x: (int(x.split('-')[0]) + int(x.split('-')[1]))/2)
df_agb = df_agb.drop('age_bucket', axis=1)
df_con = pd.read_csv('countries.csv')
df_ses = pd.read_csv('sessions.csv')
df_train = pd.read_csv('train_users_2.csv')
df_train['gender'] = df_train['gender'].replace('-unknown-', np.nan)
df_train['first_browser'] = df_train['first_browser'].replace('-unknown-', np.nan)
df_train['age'] = df_train['age'].apply(lambda x: np.nan if x > 120 else x)


#### In our EDA section, we noticed that people with signup flow 3 had a disproportionate number of conversions. Therefore, we will define an additional feature that identifies these users. We will also revert back our gender fature by filling in the NaNs with Unknown since we deduced that it will lead to an improvement in prediction.

In [8]:
df_train['is_3'] = df_train['signup_flow'].apply(lambda x: 1 if x==3 else 0)
df_train['gender'] = df_train['gender'].fillna('Unknown')

#### We will drop the first_bookings and the timsestamp features. This is because they do not add any value when finally testing the model with the test set. All our test users have NaNs as first booking and the timestamp is of no significance.



In [ ]:
df_train = first_booking(df_train)
df_train = df_train.drop('timestamp_first_active', axis=1)

#### Finally we will drop the accounts created feature as our training and test sets were separated in the middle of 2014. As a result, all test users registered only in 2014.

In [ ]:
df_train = account_created(df_train)
df_train = df_train.set_index('id')

#### Our dataset is now in a position to have one hot encoding performed on it. Let us now separate our X and y data.

In [ ]:

class_dict = {
    'NDF': 0,
    'US': 1,
    'other': 2,
    'FR': 3,
    'CA': 4,
    'GB': 5,
    'ES': 6,
    'IT': 7,
    'PT': 8,
    'NL': 9,
    'DE': 10,
    'AU': 11
}



In [ ]:
X, y = df_train.drop('country_destination', axis=1), df_train['country_destination'].apply(lambda x: class_dict[x])

X = pd.get_dummies(X, prefix='is')

train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.75, stratify=y)